# Calibration Analysis - CARDEC Trial - Any Failure (Primary Teeth)

This notebook calculates and visualizes calibration metrics for machine learning models predicting restoration failures.

**Metrics calculated:**
- Calibration Slope
- Calibration-in-the-Large (CITL)
- Observed/Expected (O/E) Ratio
- Brier Score
- Reliability Diagrams (Calibration Plots)

In [ ]:
# ============================================================================# CALIBRATION METRICS ANALYSIS# ============================================================================# This section calculates calibration metrics as requested by the reviewer:# - Calibration plots (reliability diagrams)# - Calibration slope# - Calibration-in-the-large (CITL)# - Observed/Expected (O/E) ratio# ============================================================================import numpy as npimport pandas as pdimport matplotlib.pyplot as pltfrom sklearn.calibration import calibration_curvefrom sklearn.linear_model import LogisticRegressionimport warningswarnings.filterwarnings('ignore')def calculate_calibration_metrics(y_true, y_pred_proba, model_name, n_bins=10):        # Convert to numpy arrays    y_true = np.array(y_true).ravel()    y_pred_proba = np.array(y_pred_proba).ravel()    # 1. Calculate calibration curve (for plotting)    prob_true, prob_pred = calibration_curve(y_true, y_pred_proba, n_bins=n_bins, strategy='uniform')    # 2. Calculate O/E ratio (Observed/Expected)    expected = np.sum(y_pred_proba)    observed = np.sum(y_true)    oe_ratio = observed / expected if expected > 0 else np.nan    # 3. Calculate Calibration Slope and Calibration-in-the-large (CITL)    y_pred_clipped = np.clip(y_pred_proba, 1e-10, 1 - 1e-10)    log_odds = np.log(y_pred_clipped / (1 - y_pred_clipped))    log_odds_reshaped = log_odds.reshape(-1, 1)    lr = LogisticRegression(solver='lbfgs', max_iter=1000)    lr.fit(log_odds_reshaped, y_true)    calibration_slope = lr.coef_[0][0]    citl = lr.intercept_[0]    # 4. Calculate Brier Score    brier_score = np.mean((y_pred_proba - y_true) ** 2)    return {        'model_name': model_name,        'prob_true': prob_true,        'prob_pred': prob_pred,        'calibration_slope': calibration_slope,        'citl': citl,        'oe_ratio': oe_ratio,        'brier_score': brier_score,        'observed': observed,        'expected': expected    }def plot_calibration_curves(calibration_results, dataset_name, save_path):    """    Plot calibration curves for all models.    """    fig, axes = plt.subplots(2, 3, figsize=(15, 10))    axes = axes.flatten()    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']    for idx, result in enumerate(calibration_results):        ax = axes[idx]        ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect calibration')        ax.plot(result['prob_pred'], result['prob_true'],                's-', color=colors[idx], linewidth=2, markersize=8,                label=f"{result['model_name']}")        metrics_text = (            f"Slope: {result['calibration_slope']:.3f}\n"            f"CITL: {result['citl']:.3f}\n"            f"O/E: {result['oe_ratio']:.3f}\n"            f"Brier: {result['brier_score']:.3f}"        )        ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes,                fontsize=10, verticalalignment='top',                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))        ax.set_xlabel('Mean Predicted Probability', fontsize=11)        ax.set_ylabel('Fraction of Positives', fontsize=11)        ax.set_title(f'{result["model_name"]}', fontsize=12, fontweight='bold')        ax.legend(loc='lower right', fontsize=9)        ax.set_xlim([0, 1])        ax.set_ylim([0, 1])        ax.grid(True, alpha=0.3)    ax = axes[5]    ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect calibration')    for idx, result in enumerate(calibration_results):        ax.plot(result['prob_pred'], result['prob_true'],                's-', color=colors[idx], linewidth=2, markersize=6,                label=f"{result['model_name']}")    ax.set_xlabel('Mean Predicted Probability', fontsize=11)    ax.set_ylabel('Fraction of Positives', fontsize=11)    ax.set_title('All Models Comparison', fontsize=12, fontweight='bold')    ax.legend(loc='lower right', fontsize=8)    ax.set_xlim([0, 1])    ax.set_ylim([0, 1])    ax.grid(True, alpha=0.3)    plt.suptitle(f'Calibration Plots - {dataset_name}\n(Reliability Diagrams)',                 fontsize=14, fontweight='bold', y=1.02)    plt.tight_layout()    plt.savefig(save_path, dpi=300, bbox_inches='tight')    plt.show()    print(f"\nCalibration plots saved as '{save_path}'")def print_calibration_summary(calibration_results, dataset_name, csv_filename):    """    Print a summary table of calibration metrics.    """    print("\n" + "="*80)    print(f"CALIBRATION METRICS SUMMARY - {dataset_name}")    print("="*80)    print("-"*80)    summary_data = []    for result in calibration_results:        summary_data.append({            'Model': result['model_name'],            'Calibration Slope': f"{result['calibration_slope']:.3f}",            'CITL': f"{result['citl']:.3f}",            'O/E Ratio': f"{result['oe_ratio']:.3f}",            'Brier Score': f"{result['brier_score']:.4f}",            'Observed': int(result['observed']),            'Expected': f"{result['expected']:.1f}"        })    summary_df = pd.DataFrame(summary_data)    print("\n" + summary_df.to_string(index=False))    print("-"*80)    summary_df.to_csv(csv_filename, index=False)    print(f"\nCalibration metrics saved as '{csv_filename}'")    return summary_df# ============================================================================# MAIN CALIBRATION ANALYSIS# ============================================================================DATASET_NAME = "CARDEC Trial - Any Failure (Primary Teeth)"print("\n" + "="*80)print("CALCULATING CALIBRATION METRICS FOR ALL MODELS")print("="*80)calibration_results = []# Decision Treedt_calibration = calculate_calibration_metrics(y_test, dt_avg_predictions, 'Decision Tree')calibration_results.append(dt_calibration)print("✓ Decision Tree calibration metrics calculated")# Random Forestrf_calibration = calculate_calibration_metrics(y_test, rf_avg_predictions, 'Random Forest')calibration_results.append(rf_calibration)print("✓ Random Forest calibration metrics calculated")# XGBoostxgb_calibration = calculate_calibration_metrics(y_test, xgb_avg_predictions, 'XGBoost')calibration_results.append(xgb_calibration)print("✓ XGBoost calibration metrics calculated")# CatBoostcb_calibration = calculate_calibration_metrics(y_test, cb_avg_predictions, 'CatBoost')calibration_results.append(cb_calibration)print("✓ CatBoost calibration metrics calculated")# Neural Networknn_calibration = calculate_calibration_metrics(y_test, nn_avg_predictions, 'Neural Network')calibration_results.append(nn_calibration)print("✓ Neural Network calibration metrics calculated")# Print summary tablecalibration_summary_df = print_calibration_summary(calibration_results, DATASET_NAME, 'calibration_metrics_cardec_global.csv')# Plot calibration curvesplot_calibration_curves(calibration_results, DATASET_NAME, 'calibration_plots_cardec_global.png')print("\n" + "="*80)print("CALIBRATION ANALYSIS COMPLETE")print("="*80)